In [854]:
import pandas as pd
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
import numpy as np
import re

### English Premier League

In [892]:
#transfer spending
raw_html = get("https://www.transfermarkt.us/premier-league/fuenfjahresvergleich/wettbewerb/GB1",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')

clubs = list()
for td in html.select('td'):
    clubs.append(td.text)
del clubs[:19]
del clubs[159:] #useless fluff

spending5years = dict() #total spending for every club over the past 5 years, in millions USD
for e in clubs:
    if clubs.index(e) % 8 == 0:
        spending = clubs[clubs.index(e)+6]
        spending5years[e] = float(spending[2:len(spending)-1]) #turn '$-50m' into 50
del spending5years['Norwich City']
del spending5years['Brentford FC']
del spending5years['Watford FC']
spending5years['Fulham FC'] = 160.8
spending5years['Sheffield United FC'] = 140
spending5years['West Bromwich Albion FC'] = 66.9
spending5years = pd.Series(spending5years).sort_index().to_dict()
spending5years

{'Arsenal FC': 263.67,
 'Aston Villa': 302.94,
 'Brighton & Hove Albion': 260.75,
 'Burnley FC': 26.99,
 'Chelsea FC': 256.72,
 'Crystal Palace': 23.51,
 'Everton FC': 284.81,
 'Fulham FC': 160.8,
 'Leeds United': 130.03,
 'Leicester City': 139.78,
 'Liverpool FC': 196.36,
 'Manchester City': 463.11,
 'Manchester United': 465.79,
 'Newcastle United': 120.96,
 'Sheffield United FC': 140.0,
 'Southampton FC': 53.35,
 'Tottenham Hotspur': 198.61,
 'West Bromwich Albion FC': 66.9,
 'West Ham United': 165.04,
 'Wolverhampton Wanderers': 242.26}

In [893]:
#payroll spending, average annual over 5 years if team hasn't been relegated
raw_html = get("https://www.spotrac.com/epl/payroll/2020/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
salClubs1 = list()
for td in html.select('td'):
    salClubs1.append(td.text)
salary2020 = dict() #2020-21 total payroll, in millions USD
for e in salClubs1:
    if salClubs1.index(e) % 9 == 1:
        spending = salClubs1[salClubs1.index(e)+7]
        salary2020[e.replace('.','')] = round(float(spending[1:].replace(',',''))*1.4/1000000,2)
    #*1.4 to get from pounds to usd, exchange rate accurate as of 6/23/21


raw_html = get("https://www.spotrac.com/epl/payroll/2019/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
salClubs2 = list()
for td in html.select('td'):
    salClubs2.append(td.text)
salary2019 = dict() #2019-20
for e in salClubs2:
    if salClubs2.index(e) % 9 == 1:
        spending = salClubs2[salClubs2.index(e)+7]
        salary2019[e.replace('.','')] = round(float(spending[1:].replace(',',''))*1.4/1000000,2)


raw_html = get("https://www.spotrac.com/epl/payroll/2018/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
salClubs3 = list()
for td in html.select('td'):
    salClubs3.append(td.text)
salary2018 = dict() #2018-19
for e in salClubs3:
    if salClubs3.index(e) % 9 == 1:
        spending = salClubs3[salClubs3.index(e)+7]
        salary2018[e.replace('.','')] = round(float(spending[1:].replace(',',''))*1.4/1000000,2)


raw_html = get("https://www.spotrac.com/epl/payroll/2017/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
salClubs4 = list()
for td in html.select('td'):
    salClubs4.append(td.text)
salary2017 = dict() #2017-18
for e in salClubs4:
    if salClubs4.index(e) % 9 == 1:
        spending = salClubs4[salClubs4.index(e)+7]
        salary2017[e.replace('.','')] = round(float(spending[1:].replace(',',''))*1.4/1000000,2)

raw_html = get("https://www.sportsmirchi.com/premier-league-clubs-wage-bills/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
salClubs5 = list()
for td in html.select('td'):
    salClubs5.append(td.text)
del salClubs5[:7]
del salClubs5[119:]
salary2016 = dict() #2016-17
for e in salClubs5:
    if salClubs5.index(e) % 6 == 0:
        spending = salClubs5[salClubs5.index(e)+4]
        
        if e == 'Crystal Palace' or e == 'Leicester City':
            name = e
        elif e == 'Tottenham':
            name = 'Tottenham Hotspur FC'
        elif e == 'West Bromwich':
            name = 'West Bromwich Albion FC'
        else:
            name = e+" FC"
        salary2016[name] = round(float(spending[1:-1])*1.4,2)   
        
        
#combining into dataframe to calculate averages
df = pd.DataFrame({'2016-17':pd.Series(salary2016),'2017-18':pd.Series(salary2017),'2018-19':pd.Series(salary2018),'2019-20':pd.Series(salary2019),'2020-21':pd.Series(salary2020)})
df['Average'] = df.mean(axis=1)
payRollAvrg = dict()
for index,row in df.iterrows():
    if not pd.isnull(row['2020-21']): #if team is currently in EPL
        payRollAvrg[index]=round(row['Average'],2)
payRollAvrg

{'Arsenal FC': 195.71,
 'Aston Villa FC': 48.09,
 'Brighton & Hove Albion': 57.49,
 'Burnley FC': 54.74,
 'Chelsea FC': 202.37,
 'Crystal Palace': 98.27,
 'Everton FC': 115.82,
 'Fulham FC': 59.96,
 'Leeds United FC': 31.45,
 'Leicester City': 99.2,
 'Liverpool FC': 178.85,
 'Manchester City FC': 218.33,
 'Manchester United FC': 238.64,
 'Newcastle United FC': 68.44,
 'Sheffield United FC': 23.61,
 'Southampton FC': 83.8,
 'Tottenham Hotspur FC': 141.64,
 'West Bromwich Albion FC': 67.81,
 'West Ham United FC': 106.92,
 'Wolverhampton Wanderers FC': 65.63}

In [894]:
#average league points over past 5 years
eplpoints = dict()
yearsinepl = dict() #to keep track of relegations and promotions
def eplPoints(year):
    raw_html = get("https://www.skysports.com/premier-league-table/"+year)
    html = BeautifulSoup(raw_html.content, 'html.parser')
    clubs = list()
    
    for td in html.select('td'):
        clubs.append(td.text)

    for e in clubs:
        if clubs.index(e) % 11 == 1:
            points = clubs[clubs.index(e)+8]
            name = e.strip('\n')
            if year=='2020':
                eplpoints[name] = int(points)
                yearsinepl[name] = 1
            else:
                try:
                    yearsinepl[name] += 1
                    eplpoints[name] += int(points)
                except KeyError:
                    pass #team wasn't in epl in 2020-21, not interested in it
        
    
eplPoints('2020')
eplPoints('2019')
eplPoints('2018')
eplPoints('2017')
eplPoints('2016')

for team in eplpoints.keys():
    eplpoints[team] = round(eplpoints[team]/yearsinepl[team],2)
eplpoints = pd.Series(eplpoints).sort_index().to_dict()
eplpoints

{'Arsenal': 65.0,
 'Aston Villa': 45.0,
 'Brighton and Hove Albion': 39.5,
 'Burnley': 45.4,
 'Chelsea': 73.6,
 'Crystal Palace': 44.2,
 'Everton': 54.4,
 'Fulham': 27.0,
 'Leeds United': 59.0,
 'Leicester City': 54.2,
 'Liverpool': 83.2,
 'Manchester City': 88.6,
 'Manchester United': 71.2,
 'Newcastle United': 44.5,
 'Sheffield United': 38.5,
 'Southampton': 43.2,
 'Tottenham Hotspur': 71.0,
 'West Bromwich Albion': 34.0,
 'West Ham United': 48.6,
 'Wolverhampton Wanderers': 53.67}

In [895]:
#Champions League: 1 for reaching round of 16, 2 for reaching quarterfinals, 3 for reaching semifinals, 4 for reaching finals, 5 for winning
#FA Cup: 1 for reaching 5th round proper, 2 for reaching quarterfinals, 3 for reaching semifinals, 4 for reaching finals, 5 for winning
#Europa League: 1 for reaching quarterfinals, 2 for reaching semifinals, 3 for reaching finals, 4 for winning
#EFL Cup: 1 for reaching semifinals, 2 for reaching finals, 3 for winning (since it has little prestige)

eplPlayoffs = {'Arsenal FC':46,'Aston Villa':3,'Brighton & Hove Albion':10,'Burnley FC':2,'Chelsea FC':77,
'Crystal Palace':3,'Everton FC':3,'Fulham FC':1,'Leeds United':0,'Leicester City':26,'Liverpool FC':31,'Manchester City':78,
'Manchester United':56,'Newcastle United':3,'Sheffield United':7,'Southampton FC':15,'Tottenham Hotspur':30,
'West Bromwich Albion':2,'West Ham United':1,'Wolverhampton Wanderers':9}

In [903]:
#putting it all together
#all dictionaries are sorted, but names of clubs are slightly different (e.g. Arsenal vs Arsenal FC). Therefore, index reset is required
epl = pd.DataFrame({'Total Transfer Spending Over 5 years':list(spending5years.values()),
                    'Average Annual Payroll Spending Over 5 years':list(payRollAvrg.values()),
                    'Average Season Points Over 5 years':list(eplpoints.values()),
                    'Playoff Points Over 5 years':list(eplPlayoffs.values())}, index = list(spending5years.keys()))

#plots:

# x = np.array(epl['Average Annual Payroll Spending Over 5 years'])
# y = np.array(epl['Playoff Points Over 5 years'])
# coeffs = np.polyfit(x, y, 2)
# poly = np.poly1d(coeffs)
# new_x = np.linspace(x[0]+60,x[-1]-60)
# new_y = poly(new_x)
# epl.plot(x='Average Annual Payroll Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter')
# plt.plot(new_x, new_y, color='orange')
# plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\EPLPayrollTrophies.png")


# epl.plot(x='Total Transfer Spending Over 5 years',y='Average Season Points Over 5 years',kind='scatter')
# # for index,row in epl.iterrows():
# #     plt.annotate(index,(row['Total Transfer Spending Over 5 years'],row['Average Season Points Over 5 years']))
# plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\EPLTransferPoints.png")


# epl.plot(x='Total Transfer Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter') #exponential correlation?
# plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\EPLTransferTrophies.png")


# epl.plot(x='Total Transfer Spending Over 5 years',y='Average Annual Payroll Spending Over 5 years',kind='scatter')
# epl.plot(x='Average Annual Payroll Spending Over 5 years',y='Average Season Points Over 5 years',kind='scatter')
# plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\EPLPayrollPoints.png")



# epl.to_excel(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\EPL.xlsx")
# epl

epl.corr()

,Total Transfer Spending Over 5 years,Average Annual Payroll Spending Over 5 years,Average Season Points Over 5 years,Playoff Points Over 5 years
Total Transfer Spending Over 5 years,1.000000,0.645271,0.604244,0.657808
Average Annual Payroll Spending Over 5 years,0.645271,1.000000,0.822005,0.881957
Average Season Points Over 5 years,0.604244,0.822005,1.000000,0.805457
Playoff Points Over 5 years,0.657808,0.881957,0.805457,1.000000


Conclusions:
- Payroll spending has a higher influence on success than transfer spending.
- Average points vary linearly with spending, number of major trophies vary exponentially (square).
- EPL is influenced highly by money. If you can't pay, you can't compete.

### National Football League

In [905]:
#WPCT over past 5 years
def nflPoints(year):
    WPCTdict = dict()
    raw_html = get("http://www.jt-sw.com/football/pro/standings.nsf/Seasons/"+year)
    html = BeautifulSoup(raw_html.content, 'html.parser')
    clubs = list()
    
    for td in html.select('td'):
        clubs.append(td.text)
    
    clubs = clubs[1].split('    ')
    del clubs[0]
    for i in range(len(clubs)-1,-1,-1):
        if i%5==0:
            del clubs[i]
    for string in clubs:
        string = re.split('(\d+)', string) #separate the team name from the numbers
        name = string[0].strip()
        WPCT = int(string[7])/1000
        if name == 'San Francisco':
            name = "San Francisco 49ers"
            WPCT = int(string[9])/1000
        elif name == 'Oakland Raiders':
            name = 'Las Vegas Raiders'
        elif name == "Washington Redskins":
            name = 'Washington Football Team'
        elif name == 'San Diego Chargers':
            name = 'Los Angeles Chargers'
        WPCTdict[name] = float(WPCT)     

    return WPCTdict

    
WPCT16 = nflPoints("2016")
WPCT17 = nflPoints("2017")
WPCT18 = nflPoints("2018")
WPCT19 = nflPoints("2019")
WPCT20 = nflPoints("2020")

df = pd.DataFrame({'2016':pd.Series(WPCT16),'2017':pd.Series(WPCT17),'2018':pd.Series(WPCT18),'2019':pd.Series(WPCT19),'2020':pd.Series(WPCT20)})
df['Average'] = df.mean(axis=1)
WPCTAvrg = dict()
for index,row in df.iterrows():
    WPCTAvrg[index]=round(row['Average'],3)

WPCTAvrg
#due to lack of readily available data on NFL tranfers, and the way NFL transfer system works (i.e. teams usually use draft picks or other players to pay for the transfer), there will be no tranfer expenses for NFL, only payroll spending

{'Arizona Cardinals': 0.4,
 'Atlanta Falcons': 0.488,
 'Baltimore Ravens': 0.65,
 'Buffalo Bills': 0.563,
 'Carolina Panthers': 0.425,
 'Chicago Bears': 0.45,
 'Cincinnati Bengals': 0.325,
 'Cleveland Browns': 0.319,
 'Dallas Cowboys': 0.575,
 'Denver Broncos': 0.4,
 'Detroit Lions': 0.407,
 'Green Bay Packers': 0.619,
 'Houston Texans': 0.475,
 'Indianapolis Colts': 0.5,
 'Jacksonville Jaguars': 0.313,
 'Kansas City Chiefs': 0.75,
 'Las Vegas Raiders': 0.463,
 'Los Angeles Chargers': 0.475,
 'Los Angeles Rams': 0.588,
 'Miami Dolphins': 0.475,
 'Minnesota Vikings': 0.581,
 'New England Patriots': 0.713,
 'New Orleans Saints': 0.7,
 'New York Giants': 0.363,
 'New York Jets': 0.288,
 'Philadelphia Eagles': 0.532,
 'Pittsburgh Steelers': 0.669,
 'San Francisco 49ers': 0.388,
 'Seattle Seahawks': 0.656,
 'Tampa Bay Buccaneers': 0.463,
 'Tennessee Titans': 0.588,
 'Washington Football Team': 0.407}

In [906]:
#Payroll spending over past 5 years
raw_html = get("https://overthecap.com/positional-spending/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
pay = list()
for td in html.select('td'):
    pay.append(td.text)

def getPayroll(lis):
    cap = dict()
    for e in lis:
        if lis.index(e)%13==0:
            for i in WPCT18.keys():
                if e in i:
                    name = i
            off = lis[lis.index(e)+6] #offense spending
            de = lis[lis.index(e)+12] #defense spending
            spending = round((float(off[1:].replace(',',''))+float(de[1:].replace(',','')))/1000000,2)
            cap[name]=spending
    return cap

del pay[:1248] #pre-2016
pay16 = pay[:416]
pay17 = pay[416:832]
pay18 = pay[832:1248]
pay19 = pay[1248:1664]
pay20 = pay[1664:2080]

cap16 = getPayroll(pay16)
cap17 = getPayroll(pay17)
cap18 = getPayroll(pay18)
cap19 = getPayroll(pay19)
cap20 = getPayroll(pay20)


df1 = pd.DataFrame({'2016':pd.Series(cap16),'2017':pd.Series(cap17),'2018':pd.Series(cap18),'2019':pd.Series(cap19),'2020':pd.Series(cap20)})
df1['Average'] = df1.mean(axis=1)
capAvrg = dict()
for index,row in df1.iterrows():
    capAvrg[index]=round(row['Average'],2)
capAvrg

{'Arizona Cardinals': 146.9,
 'Atlanta Falcons': 152.28,
 'Baltimore Ravens': 142.77,
 'Buffalo Bills': 141.76,
 'Carolina Panthers': 148.37,
 'Chicago Bears': 159.12,
 'Cincinnati Bengals': 161.81,
 'Cleveland Browns': 142.07,
 'Dallas Cowboys': 146.77,
 'Denver Broncos': 146.68,
 'Detroit Lions': 144.62,
 'Green Bay Packers': 161.41,
 'Houston Texans': 155.44,
 'Indianapolis Colts': 151.15,
 'Jacksonville Jaguars': 150.72,
 'Kansas City Chiefs': 152.76,
 'Las Vegas Raiders': 158.67,
 'Los Angeles Chargers': 157.97,
 'Los Angeles Rams': 147.69,
 'Miami Dolphins': 141.49,
 'Minnesota Vikings': 159.68,
 'New England Patriots': 149.99,
 'New Orleans Saints': 139.81,
 'New York Giants': 143.93,
 'New York Jets': 139.98,
 'Philadelphia Eagles': 151.45,
 'Pittsburgh Steelers': 154.25,
 'San Francisco 49ers': 148.05,
 'Seattle Seahawks': 152.58,
 'Tampa Bay Buccaneers': 165.12,
 'Tennessee Titans': 157.44,
 'Washington Football Team': 151.0}

In [907]:
#A team is assigned points based on how far they have gone in the playoffs: 1 for reaching wildcard round, 2 for divisional playoffs, 3 for conference championship, 4 for the final, 5 for winning superbowl.
#These points  add up through rounds: a superbowl winner will get 15 points (1+2+3+4+5), not just 5
#Teams that got to the divisional playoffs without playing in the wildcard round get 3 points

nflPlayoffs = {'Arizona Cardinals':0,'Atlanta Falcons':13,'Baltimore Ravens':7,'Buffalo Bills':8,'Carolina Panthers':1,
'Chicago Bears':2,'Cincinnati Bengals':0,'Cleveland Browns':3,'Dallas Cowboys':6,'Denver Broncos':0,'Detroit Lions':1,
'Green Bay Packers':18,'Houston Texans':7,'Indianapolis Colts':4,'Jacksonville Jaguars':6,'Kansas City Chiefs':35,
'Las Vegas Raiders':1,'Los Angeles Chargers':3,'Los Angeles Rams':14,'Miami Dolphins':1,'Minnesota Vikings':9,
'New England Patriots':41,'New Orleans Saints':13,'New York Giants':1,'New York Jets':0,
'Philadelphia Eagles':19,'Pittsburgh Steelers':10,'San Francisco 49ers':10,'Seattle Seahawks':8,'Tampa Bay Buccaneers':15,
'Tennessee Titans':10,'Washington Football Team':1}

In [908]:
#putting it all together
nfl = pd.DataFrame({'Average Annual Payroll Spending Over 5 years':pd.Series(capAvrg),
                    'Average WPCT Over 5 years':pd.Series(WPCTAvrg),
                    'Playoff Points Over 5 years':pd.Series(nflPlayoffs)})

#plots:
# nfl.plot(x='Average Annual Payroll Spending Over 5 years',y='Average WPCT Over 5 years',kind='scatter') #most teams spend the same due to spending cap
# plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\NFLWPCT.png")

# nfl.plot(x='Average Annual Payroll Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter')
# plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\NFLPlayoffs.png")

# nfl.plot(x='Average WPCT Over 5 years',y='Playoff Points Over 5 years',kind='scatter') #strong exponential correlation since many of the trophies depend on WPCT

nfl.corr()
#nfl
#nfl.to_excel(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\NFL.xlsx")

,Average Annual Payroll Spending Over 5 years,Average WPCT Over 5 years,Playoff Points Over 5 years
Average Annual Payroll Spending Over 5 years,1.000000,0.097603,0.167660
Average WPCT Over 5 years,0.097603,1.000000,0.699372
Playoff Points Over 5 years,0.167660,0.699372,1.000000


Conclusions:
- Spending barely has an influence on success. This is most likely due to the spending cap, as well as the draft system. Football is also a very coordinated team sport, so individual expensive players do not have that much influence.

### Major League Soccer

In [909]:
#transfer spending. Some of the teams are new and haven't been in MLS for all 5 years, so it's average spending per year, not total

raw_html = get("https://www.transfermarkt.com/major-league-soccer/fuenfjahresvergleich/wettbewerb/MLS1",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
mlsTrans = list()
for td in html.select('td'):
    mlsTrans.append(td.text)
del mlsTrans[:18]
del mlsTrans[215:]

#how many seasons out of the past 5 seasons every team missed
#LAFC: 1, Cincinnati: 2, Miami and Nashville: 3, Austin: 4
#some teams in transfermarkt table have "-" for certain seasons because they have paid $0 in transfers that season. This doesn't count as a missed season

mlsTransferAvrg = dict() #average spending for every club over the past 5 years, in millions USD

for e in mlsTrans:
    if mlsTrans.index(e) % 8 == 0:
        spending = mlsTrans[mlsTrans.index(e)+6]
        yearsInMLS = 5
        if e == 'Los Angeles FC':
            yearsInMLS = 4
        elif e == 'FC Cincinnati':
            yearsInMLS = 3
        elif e == 'Inter Miami CF' or e == 'Nashville SC':
            yearsInMLS = 2
        elif e == 'Austin FC':
            continue #Austin is new this season, so it doesn't yet have a point average or trophies
        if '-' in spending:
            spending = float(spending[2:len(spending)-1])*1.19 #turn '$-50m' into 50
            #*1.19 to get from euros to usd, exchange rate accurate as of 6/29/21
        else: #some teams have actually earned money on the net
            spending = -float(spending[1:len(spending)-1])*1.19 #turn '$50m' into 50
            
        mlsTransferAvrg[e] = round(spending/yearsInMLS,2)
#mlsTrans
mlsTransferAvrg = pd.Series(mlsTransferAvrg).sort_index().to_dict()
mlsTransferAvrg

{'Atlanta United FC': 4.73,
 'Chicago Fire FC': 3.8,
 'Club de Foot Montréal': 1.05,
 'Colorado Rapids': 0.82,
 'Columbus Crew': 1.51,
 'D.C. United': 2.8,
 'FC Cincinnati': 9.44,
 'FC Dallas': -2.11,
 'Houston Dynamo FC': 0.63,
 'Inter Miami CF': 17.19,
 'Los Angeles FC': 8.07,
 'Los Angeles Galaxy': 5.07,
 'Minnesota United FC': 3.78,
 'Nashville SC': 4.37,
 'New England Revolution': 2.78,
 'New York City FC': 4.05,
 'New York Red Bulls': 1.76,
 'Orlando City SC': 0.91,
 'Philadelphia Union': -1.4,
 'Portland Timbers': 5.49,
 'Real Salt Lake City': 0.67,
 'San Jose Earthquakes': 0.78,
 'Seattle Sounders FC': 3.76,
 'Sporting Kansas City': 3.2,
 'Toronto FC': 3.36,
 'Vancouver Whitecaps FC': 1.25}

In [910]:
#payroll spending, average over past 5 years
raw_html = get("https://www.spotrac.com/mls/cap/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
salClubs1 = list()
for td in html.select('td'):
    salClubs1.append(td.text)
salary2021 = dict() #2021 total payroll, in millions USD
for e in salClubs1:
    if salClubs1.index(e) % 7 == 1:
        spending = salClubs1[salClubs1.index(e)+5]
        if e == "Montreal Impact":
            e = "CF Montreal"
        if e == 'Austin FC':
            continue
        salary2021[e] = round(float(spending[1:].replace(',',''))/1000000,2)

raw_html = get("https://www.spotrac.com/mls/cap/2020",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
salClubs2 = list()
for td in html.select('td'):
    salClubs2.append(td.text)
salary2020 = dict() #2020 total payroll, in millions USD
for e in salClubs2:
    if salClubs2.index(e) % 7 == 1:
        spending = salClubs2[salClubs2.index(e)+5]
        if e == "Montreal Impact":
            e = "CF Montreal"
        salary2020[e] = round(float(spending[1:].replace(',',''))/1000000,2)
        
raw_html = get("https://www.spotrac.com/mls/cap/2019",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
salClubs3 = list()
for td in html.select('td'):
    salClubs3.append(td.text)
salary2019 = dict() #2019 total payroll, in millions USD
for e in salClubs3:
    if salClubs3.index(e) % 7 == 1:
        spending = salClubs3[salClubs3.index(e)+5]
        if e == "Montreal Impact":
            e = "CF Montreal"
        salary2019[e] = round(float(spending[1:].replace(',',''))/1000000,2)

raw_html = get("https://www.spotrac.com/mls/cap/2018",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
salClubs4 = list()
for td in html.select('td'):
    salClubs4.append(td.text)
salary2018 = dict() #2018 total payroll, in millions USD
for e in salClubs4:
    if salClubs4.index(e) % 7 == 1:
        spending = salClubs4[salClubs4.index(e)+5]
        if e == "Montreal Impact":
            e = "CF Montreal"
        salary2018[e] = round(float(spending[1:].replace(',',''))/1000000,2)


#no easy scrape sources for 2017, so I'll do it manually
salary2017 = {'CF Montreal': 6.32,'Chicago Fire': 13.09,'Colorado Rapids': 8.26,'Columbus Crew': 6.77,'D.C. United': 5.42,
 'FC Dallas': 7.35,'Houston Dynamo': 5.77,'LA Galaxy': 14.57,'Minnesota United FC': 5.75,'New England Revolution': 7.5,
 'New York City FC': 18.11,'New York Red Bulls': 7.29,'Orlando City': 14.58,'Philadelphia Union': 7.18,'Portland Timbers': 11.37,
 'Real Salt Lake': 8.58,'San Jose Earthquakes': 8.44,'Seattle Sounders FC': 11.82,'Sporting Kansas City': 6.3,
 'Toronto FC': 22.55,'Vancouver Whitecaps FC': 8.44}
        
#combining into dataframe to calculate averages
df = pd.DataFrame({'2017':pd.Series(salary2017),'2018':pd.Series(salary2018),'2019':pd.Series(salary2019),'2020':pd.Series(salary2020),'2021':pd.Series(salary2021)})
df['Average'] = df.mean(axis=1)
payRollAvrg = dict()
for index,row in df.iterrows():
    payRollAvrg[index]=round(row['Average'],2)
payRollAvrg

{'Atlanta United FC': 12.33,
 'CF Montreal': 10.08,
 'Chicago Fire': 13.32,
 'Colorado Rapids': 8.29,
 'Columbus Crew': 9.77,
 'D.C. United': 8.45,
 'FC Cincinnati': 9.52,
 'FC Dallas': 8.46,
 'Houston Dynamo': 7.22,
 'Inter Miami FC': 14.66,
 'LA Galaxy': 16.3,
 'Los Angeles FC': 13.62,
 'Minnesota United FC': 8.22,
 'Nashville SC': 6.28,
 'New England Revolution': 9.09,
 'New York City FC': 13.2,
 'New York Red Bulls': 7.64,
 'Orlando City': 10.64,
 'Philadelphia Union': 8.02,
 'Portland Timbers': 11.04,
 'Real Salt Lake': 9.17,
 'San Jose Earthquakes': 8.58,
 'Seattle Sounders FC': 12.11,
 'Sporting Kansas City': 10.49,
 'Toronto FC': 20.72,
 'Vancouver Whitecaps FC': 7.38}

In [911]:
#average league points over past 5 years. 2020 didn't have all the teams play 34 games as usual due to covid, so the points dropped. Not sure what to do with it, I included it
raw_html = get("https://www.transfermarkt.us/major-league-soccer/tabelle/wettbewerb/MLS1?saison_id=2019",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
pClubs1 = list()
for td in html.select('td'):
    pClubs1.append(td.text)

del pClubs1[:13]
del pClubs1[138:141]
del pClubs1[259:]

points20 = dict() #points in 2020
for e in pClubs1:
    if pClubs1.index(e) % 10 == 0:
        points = pClubs1[pClubs1.index(e)+7]
        e = e.strip('\n \t') 
        if e == 'Montreal Impact': #to make it properly alphabetically sorted to join into a single dataframe in the end
            e = 'CF Montreal'
        if e == 'Cincinnati':
            e = 'FC Cincinnati'
        if e == 'Revolution':
            e = 'New England Revolution'
        if e == 'Red Bulls':
            e = 'New York Red Bulls'
        if e == 'NYCFC':
            e = 'New York City FC'
        if e == 'Quakes':
            e = 'San Jose Earthquakes'
        if e == 'Timbers':
            e = 'Portland Timbers'
        if e == 'Salt Lake':
            e = 'Real Salt Lake'
        points20[e] = int(points)

raw_html = get("https://www.transfermarkt.us/major-league-soccer/tabelle/wettbewerb/MLS1?saison_id=2018",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
pClubs2 = list()
for td in html.select('td'):
    pClubs2.append(td.text)
del pClubs2[:13]
del pClubs2[118:121]
del pClubs2[240:]

points19 = dict() #points in 2019
for e in pClubs2:
    if pClubs2.index(e) % 10 == 0:
        points = pClubs2[pClubs2.index(e)+7]
        e = e.strip('\n \t') 
        if e == 'Montreal Impact':
            e = 'CF Montreal'
        if e == 'Cincinnati':
            e = 'FC Cincinnati'
        if e == 'Revolution':
            e = 'New England Revolution'
        if e == 'Red Bulls':
            e = 'New York Red Bulls'
        if e == 'NYCFC':
            e = 'New York City FC'
        if e == 'Quakes':
            e = 'San Jose Earthquakes'
        if e == 'Timbers':
            e = 'Portland Timbers'
        if e == 'Salt Lake':
            e = 'Real Salt Lake'
        points19[e] = int(points)

        
raw_html = get("https://www.transfermarkt.us/major-league-soccer/tabelle/wettbewerb/MLS1?saison_id=2017",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
pClubs3 = list()
for td in html.select('td'):
    pClubs3.append(td.text)
del pClubs3[:13]
del pClubs3[108:111]
del pClubs3[230:]
points18 = dict() #points in 2018
for e in pClubs3:
    if pClubs3.index(e) % 10 == 0:
        points = pClubs3[pClubs3.index(e)+7]
        e = e.strip('\n \t') 
        if e == 'Montreal Impact':
            e = 'CF Montreal'
        if e == 'Cincinnati':
            e = 'FC Cincinnati'
        if e == 'Revolution':
            e = 'New England Revolution'
        if e == 'Red Bulls':
            e = 'New York Red Bulls'
        if e == 'NYCFC':
            e = 'New York City FC'
        if e == 'Quakes':
            e = 'San Jose Earthquakes'
        if e == 'Timbers':
            e = 'Portland Timbers'
        if e == 'Salt Lake':
            e = 'Real Salt Lake'
        points18[e] = int(points)

        
raw_html = get("https://www.transfermarkt.us/major-league-soccer/tabelle/wettbewerb/MLS1?saison_id=2016",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
pClubs4 = list()
for td in html.select('td'):
    pClubs4.append(td.text)
del pClubs4[:13]
del pClubs4[108:111]
del pClubs4[220:]

points17 = dict() #points in 2017
for e in pClubs4:
    if pClubs4.index(e) % 10 == 0:
        points = pClubs4[pClubs4.index(e)+7]
        e = e.strip('\n \t') 
        if e == 'Montreal Impact':
            e = 'CF Montreal'
        if e == 'Cincinnati':
            e = 'FC Cincinnati'
        if e == 'Revolution':
            e = 'New England Revolution'
        if e == 'Red Bulls':
            e = 'New York Red Bulls'
        if e == 'NYCFC':
            e = 'New York City FC'
        if e == 'Quakes':
            e = 'San Jose Earthquakes'
        if e == 'Timbers':
            e = 'Portland Timbers'
        if e == 'Salt Lake':
            e = 'Real Salt Lake'
        points17[e] = int(points)


raw_html = get("https://www.transfermarkt.us/major-league-soccer/tabelle/wettbewerb/MLS1?saison_id=2015",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
pClubs5 = list()
for td in html.select('td'):
    pClubs5.append(td.text)
del pClubs5[:13]
del pClubs5[98:101]
del pClubs5[200:]

points16 = dict() #points in 2016
for e in pClubs5:
    if pClubs5.index(e) % 10 == 0:
        points = pClubs5[pClubs5.index(e)+7]
        e = e.strip('\n \t') 
        if e == 'Montreal Impact':
            e = 'CF Montreal'
        if e == 'Cincinnati':
            e = 'FC Cincinnati'
        if e == 'Revolution':
            e = 'New England Revolution'
        if e == 'Red Bulls':
            e = 'New York Red Bulls'
        if e == 'NYCFC':
            e = 'New York City FC'
        if e == 'Quakes':
            e = 'San Jose Earthquakes'
        if e == 'Timbers':
            e = 'Portland Timbers'
        if e == 'Salt Lake':
            e = 'Real Salt Lake'
        points16[e] = int(points)

#combining into dataframe to calculate averages
df = pd.DataFrame({'2016':pd.Series(points16),'2017':pd.Series(points17),'2018':pd.Series(points18),'2019':pd.Series(points19),'2020':pd.Series(points20)})
df['Average'] = df.mean(axis=1)
pointAvrg = dict()
for index,row in df.iterrows():
    pointAvrg[index]=round(row['Average'],2)
pointAvrg

{'Atlanta United': 51.0,
 'CF Montreal': 39.4,
 'Chicago Fire': 36.6,
 'Colorado': 38.4,
 'Columbus Crew': 44.0,
 'D.C. United': 40.0,
 'FC Cincinnati': 20.0,
 'FC Dallas': 49.0,
 'Houston': 36.6,
 'Inter Miami CF': 26.0,
 'LA Galaxy': 41.0,
 'Los Angeles FC': 53.67,
 'Minnesota Utd.': 39.75,
 'Nashville SC': 32.0,
 'New England Revolution': 41.0,
 'New York City FC': 54.0,
 'New York Red Bulls': 51.6,
 'Orlando City SC': 37.2,
 'Philadelphia': 47.2,
 'Portland Timbers': 47.8,
 'Real Salt Lake': 43.0,
 'San Jose Earthquakes': 35.8,
 'Sounders FC': 51.0,
 'Sporting KC': 47.0,
 'Toronto FC': 50.4,
 'Vancouver': 39.8}

In [919]:
#A team is assigned points based on how far they have gone in the playoffs: 
#MLS Cup: 1 for reaching first round, 2 for conference semifinals, 3 for conference finals, 4 for the final, 5 for winning mls cup.
#US Open Cup: 1 for reaching round of 16, 2 for quarterfinals, 3 for semifinals, 4 for the final, 5 for winning US open cup.
#Concacaf Champions league: 1 for reaching round of 16 (if available), 2 for reaching quarterfinals, 3 for semifinals, 4 for the final, 5 for winning Champions League.

#These points  add up through rounds: an mls cup winner will get 15 points (1+2+3+4+5), not just 5
#Teams that got to conference semifinals without playign in first round get +3
mlsplayoffPoints = {'Atlanta United FC': 45,'CF Montreal': 9,'Chicago Fire': 14,'Colorado Rapids': 10,
 'Columbus Crew': 26,'D.C. United': 8,'FC Cincinnati': 7,'FC Dallas': 34,'Houston Dynamo': 29,'Inter Miami FC': 0,
 'LA Galaxy': 19,'Los Angeles FC': 28,'Minnesota United FC': 18,'Nashville SC': 4,'New England Revolution': 21,
 'New York City FC': 19,'New York Red Bulls': 37,'Orlando City': 13,'Philadelphia Union': 20,'Portland Timbers': 25,
 'Real Salt Lake': 10,'San Jose Earthquakes': 9,'Seattle Sounders FC': 63,'Sporting Kansas City': 36,'Toronto FC': 47,'Vancouver Whitecaps FC': 8}

In [920]:
#putting it all together
#all dictionaries are sorted, but names of clubs are slightly different (e.g. Atlanta United vs Atlanta United FC). Therefore, index reset is required
mls = pd.DataFrame({'Average Annual Transfer Spending Over 5 years':list(mlsTransferAvrg.values()),
                    'Average Annual Payroll Spending Over 5 years':list(payRollAvrg.values()),
                    'Average Season Points Over 5 years':list(pointAvrg.values()),
                    'Playoff Points Over 5 years':list(mlsplayoffPoints.values())}, index = list(mlsplayoffPoints.keys()))

mls = mls.drop('Inter Miami FC') #outlier. Spent 17 million on transfers on average, 14 million on payroll, but no trophies and low points (only 26)
#mls = mls.drop('FC Cincinnati')
#mls
#mls.to_excel(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\MLS.xlsx")


#plots:
#mls.plot(x='Average Annual Transfer Spending Over 5 years',y='Average Season Points Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\MLSTransferPoints.png")

#mls.plot(x='Average Annual Transfer Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\MLSTransferPlayoffs.png")

#mls.plot(x='Average Annual Transfer Spending Over 5 years',y='Average Annual Payroll Spending Over 5 years',kind='scatter')

#mls.plot(x='Average Annual Payroll Spending Over 5 years',y='Average Season Points Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\MLSPayrollPoints.png")

#mls.plot(x='Average Annual Payroll Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\MLSPayrollPlayoffs.png")

mls.corr()

,Average Annual Transfer Spending Over 5 years,Average Annual Payroll Spending Over 5 years,Average Season Points Over 5 years,Playoff Points Over 5 years
Average Annual Transfer Spending Over 5 years,1.000000,0.391980,-0.202258,0.002498
Average Annual Payroll Spending Over 5 years,0.391980,1.000000,0.381793,0.431157
Average Season Points Over 5 years,-0.202258,0.381793,1.000000,0.683945
Playoff Points Over 5 years,0.002498,0.431157,0.683945,1.000000


Conclusions:
- In MLS, spending is much less important for success than in EPL. This is likely caused by the draft system and caps on salaries, as well as by the fact that good players will be reluctant to play in MLS even if offered large transfer fees.
- Transfer spending barely has an impact on trophies. Suprisingly, it has a slight negative correlation with points, suggesting that teams spending less on transfers may actually be more successful than their high-spending counterparts.
- Payroll, meanwhile, does play a role in a team's success. Generally, the more a team spends, the more trophies and points they will get, though this relationship is not as pronounced as in EPL.

### National Basketball Association

In [914]:
#no transfer spending - only payroll (total, not average)
nbaPayroll = dict()
def addSal(year):
    raw_html = get("https://hoopshype.com/salaries/"+year,headers = {"User-Agent":"Mozilla/5.0"})
    html = BeautifulSoup(raw_html.content, 'html.parser')
    clubs = list()
    for td in html.select('td'):
        clubs.append(td.text)
    if year=='':
        del clubs[:9]
        for i in clubs:
            if clubs.index(i) % 8 == 0:
                club = i.strip('\t \n')
                spending = clubs[clubs.index(i)+1].strip('\t \n')
                spending = round(float(spending[1:].replace(',',''))/1000000,2)
                nbaPayroll[club]=spending
    else:
        del clubs[:5]
        for i in clubs:
            if clubs.index(i) % 4 == 0:
                club = i.strip('\t \n')
                spending = clubs[clubs.index(i)+1].strip('\t \n')
                spending = float(spending[1:].replace(',',''))/1000000
                nbaPayroll[club]+=spending
                nbaPayroll[club] = round(nbaPayroll[club],2)

addSal('')
addSal('2019-2020')
addSal('2018-2019')
addSal('2017-2018')
addSal('2016-2017')
nbaPayroll = pd.Series(nbaPayroll).sort_index().to_dict()
nbaPayroll

{'Atlanta': 479.89,
 'Boston': 579.14,
 'Brooklyn': 584.74,
 'Charlotte': 546.63,
 'Chicago': 537.15,
 'Cleveland': 648.82,
 'Dallas': 523.13,
 'Denver': 568.15,
 'Detroit': 574.39,
 'Golden State': 685.84,
 'Houston': 597.23,
 'Indiana': 539.38,
 'LA Clippers': 623.11,
 'LA Lakers': 563.15,
 'Memphis': 577.41,
 'Miami': 652.13,
 'Milwaukee': 605.84,
 'Minnesota': 566.1,
 'New Orleans': 590.25,
 'New York': 533.75,
 'Oklahoma City': 604.1,
 'Orlando': 565.36,
 'Philadelphia': 578.45,
 'Phoenix': 518.52,
 'Portland': 632.58,
 'Sacramento': 517.31,
 'San Antonio': 590.28,
 'Toronto': 614.91,
 'Utah': 554.62,
 'Washington': 603.9}

In [915]:
#average WPCT over 5 seasons
raw_html = get("https://www.basketball-reference.com/tools/share.fcgi?id=sxCLG",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
nba1 = list()
for td in html.select('td'):
    nba1.append(td.text)
WPCT2021 = dict() #2020-2021
for i in nba1:
    if nba1.index(i)%2==0:
        record = nba1[nba1.index(i)+1].split('-')
        wins = int(record[0])
        losses = int(record[1])
        WPCT2021[i] = round(wins/(wins+losses),3)
        

raw_html = get("https://www.basketball-reference.com/tools/share.fcgi?id=pHemP",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
nba2 = list()
for td in html.select('td'):
    nba2.append(td.text)
WPCT2020 = dict() #2019-2020
for i in nba2:
    if nba2.index(i)%2==0:
        record = nba2[nba2.index(i)+1].split('-')
        wins = int(record[0])
        losses = int(record[1])
        WPCT2020[i] = round(wins/(wins+losses),3)

raw_html = get("https://www.basketball-reference.com/tools/share.fcgi?id=cEUbZ",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
nba3 = list()
for td in html.select('td'):
    nba3.append(td.text)
WPCT2019 = dict() #2018-2019
for i in nba3:
    if nba3.index(i)%2==0:
        record = nba3[nba3.index(i)+1].split('-')
        wins = int(record[0])
        losses = int(record[1])
        WPCT2019[i] = round(wins/(wins+losses),3)

        
raw_html = get("https://www.basketball-reference.com/tools/share.fcgi?id=BUtEm",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
nba4 = list()
for td in html.select('td'):
    nba4.append(td.text)
WPCT2018 = dict() #2017-2018
for i in nba4:
    if nba4.index(i)%2==0:
        record = nba4[nba4.index(i)+1].split('-')
        wins = int(record[0])
        losses = int(record[1])
        WPCT2018[i] = round(wins/(wins+losses),3)


raw_html = get("https://www.basketball-reference.com/tools/share.fcgi?id=D0rAs",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
nba5 = list()
for td in html.select('td'):
    nba5.append(td.text)
WPCT2017 = dict() #2016-2017
for i in nba5:
    if nba5.index(i)%2==0:
        record = nba5[nba5.index(i)+1].split('-')
        wins = int(record[0])
        losses = int(record[1])
        WPCT2017[i] = round(wins/(wins+losses),3)

        
df = pd.DataFrame({'2016-17':pd.Series(WPCT2017),'2017-18':pd.Series(WPCT2018),'2018-19':pd.Series(WPCT2019),'2019-20':pd.Series(WPCT2020),'2020-21':pd.Series(WPCT2021)})
df['Average'] = df.mean(axis=1)
WPCTAvrg = dict()
for index,row in df.iterrows():
    WPCTAvrg[index]=round(row['Average'],3)

WPCTAvrg

{'Atlanta Hawks': 0.408,
 'Boston Celtics': 0.616,
 'Brooklyn Nets': 0.45,
 'Charlotte Hornets': 0.433,
 'Chicago Bulls': 0.373,
 'Cleveland Cavaliers': 0.412,
 'Dallas Mavericks': 0.451,
 'Denver Nuggets': 0.598,
 'Detroit Pistons': 0.402,
 'Golden State Warriors': 0.598,
 'Houston Rockets': 0.591,
 'Indiana Pacers': 0.554,
 'Los Angeles Clippers': 0.611,
 'Los Angeles Lakers': 0.502,
 'Memphis Grizzlies': 0.438,
 'Miami Heat': 0.534,
 'Milwaukee Bucks': 0.637,
 'Minnesota Timberwolves': 0.401,
 'New Orleans Pelicans': 0.45,
 'New York Knicks': 0.365,
 'Oklahoma City Thunder': 0.535,
 'Orlando Magic': 0.383,
 'Philadelphia 76ers': 0.573,
 'Phoenix Suns': 0.391,
 'Portland Trail Blazers': 0.56,
 'Sacramento Kings': 0.411,
 'San Antonio Spurs': 0.562,
 'Toronto Raptors': 0.632,
 'Utah Jazz': 0.63,
 'Washington Wizards': 0.466}

In [916]:
#A team is assigned points based on how far they have gone in the playoffs: 
#1 for reaching first round, 2 for conference semifinals, 3 for conference finals, 4 for the final, 5 for winning nba.

nbaPlayoffs = {'Atlanta Hawks': 4,'Boston Celtics': 22,'Brooklyn Nets': 2,'Charlotte Hornets': 1,'Chicago Bulls': 1,
'Cleveland Cavaliers': 35,'Dallas Mavericks': 2,'Denver Nuggets': 9,'Detroit Pistons': 2,'Golden State Warriors': 50,
'Houston Rockets': 16,'Indiana Pacers': 5,'Los Angeles Clippers': 6,'Los Angeles Lakers': 15,'Memphis Grizzlies': 2,
'Miami Heat': 14,'Milwaukee Bucks': 11,'Minnesota Timberwolves': 1,'New Orleans Pelicans': 3,'New York Knicks': 0,
'Oklahoma City Thunder': 10,'Orlando Magic': 2,'Philadelphia 76ers': 7,'Phoenix Suns': 0,'Portland Trail Blazers': 12,
'Sacramento Kings': 0,'San Antonio Spurs': 11,'Toronto Raptors': 30,'Utah Jazz': 8,'Washington Wizards': 4}

In [917]:
#putting it all together
#all dictionaries are sorted, but names of clubs are slightly different (e.g. New York vs New York Knicks). Therefore, index reset is required
nba = pd.DataFrame({'Total Payroll Spending Over 5 years':list(nbaPayroll.values()),
                    'Average WPCT Over 5 years':list(WPCTAvrg.values()),
                    'Playoff Points Over 5 years':list(nbaPlayoffs.values())}, index = list(nbaPlayoffs.keys()))

#plots:
#nba.plot(x='Total Payroll Spending Over 5 years',y='Average WPCT Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\NBAWPCT.png")

#nba.plot(x='Total Payroll Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\NBAPlayoffs.png")

nba.corr()

#nba
#nba.to_excel(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\NBA.xlsx")

,Total Payroll Spending Over 5 years,Average WPCT Over 5 years,Playoff Points Over 5 years
Total Payroll Spending Over 5 years,1.000000,0.496402,0.707794
Average WPCT Over 5 years,0.496402,1.000000,0.503947
Playoff Points Over 5 years,0.707794,0.503947,1.000000


Conclusions:
- Surprisingly, there is a fairly strong correlation between spending and both trophies and WPCT. It is certainly much stronger than NFL (where it was virtually nonexistent), despite having similar draft and cap systems. This could be caused by multiple factors, such as
    - Less restrictive cap policies (NBA allows teams to go over the cap as long as they pay a tax, whereas NFL has a hard cap).
    - Nature of the sport (individual prowess is more important in basketball than in football, where strategy and teamwork are far more significant).
    - NFL season only has 16 games and playoff are single-elimination, so the role of luck is increased.
    - Higher risk of injuries in NFL adds uncertainty into the relationship between payroll and success. 

  
  Nevertheless, the NBA correlation between spending and success is not as strong as EPL.

### National Hockey League

In [918]:
#payroll spending, average over 5 years (since Las Vegas joined in 2017, can't use total over 5 years)
nhlPayroll = dict()
def nhlSal(year):
    raw_html = get("https://www.capfriendly.com/archive/"+year,headers = {"User-Agent":"Mozilla/5.0"})
    html = BeautifulSoup(raw_html.content, 'html.parser')
    clubs = list()
    #print(html)
    for td in html.select('td'):
        clubs.append(td.text)
    
    for i in clubs:
        if clubs.index(i) % 4 == 0:
            spending = clubs[clubs.index(i)+1]
            if not i in nhlPayroll.keys():
                nhlPayroll[i] = float(spending[1:].replace(',',''))/1000000
            else:
                nhlPayroll[i] += float(spending[1:].replace(',',''))/1000000

nhlSal("2021")
nhlSal("2020")
nhlSal("2019")
nhlSal("2018")
nhlSal("2017")

for team in nhlPayroll.keys():
    if team == 'Vegas Golden Knights':
        nhlPayroll[team] = round(nhlPayroll[team]/4,2)
    else:
        nhlPayroll[team] = round(nhlPayroll[team]/5,2)

        nhlPayroll = pd.Series(nhlPayroll).sort_index().to_dict()
nhlPayroll

{'Anaheim Ducks': 78.64,
 'Arizona Coyotes': 74.48,
 'Boston Bruins': 76.92,
 'Buffalo Sabres': 76.15,
 'Calgary Flames': 77.47,
 'Carolina Hurricanes': 68.22,
 'Chicago Blackhawks': 78.77,
 'Colorado Avalanche': 73.23,
 'Columbus Blue Jackets': 74.98,
 'Dallas Stars': 78.49,
 'Detroit Red Wings': 79.22,
 'Edmonton Oilers': 76.35,
 'Florida Panthers': 73.95,
 'Los Angeles Kings': 73.85,
 'Minnesota Wild': 76.68,
 'Montreal Canadiens': 74.24,
 'Nashville Predators': 75.37,
 'New Jersey Devils': 68.07,
 'New York Islanders': 76.45,
 'New York Rangers': 74.66,
 'Ottawa Senators': 71.6,
 'Philadelphia Flyers': 76.3,
 'Pittsburgh Penguins': 79.26,
 'San Jose Sharks': 77.02,
 'St. Louis Blues': 79.86,
 'Tampa Bay Lightning': 80.59,
 'Toronto Maple Leafs': 81.71,
 'Vancouver Canucks': 77.6,
 'Vegas Golden Knights': 77.64,
 'Washington Capitals': 79.0,
 'Winnipeg Jets': 74.86}

In [921]:
#average points over 5 years
nhlpAvrg = dict()
def nhlPoints(year):
    if year == '2021':
        raw_html = get("https://www.espn.com/nhl/standings/_/group/league",headers = {"User-Agent":"Mozilla/5.0"})
    else:
        raw_html = get("https://www.espn.com/nhl/standings/_/season/"+year+"/group/league")
    html = BeautifulSoup(raw_html.content, 'html.parser')
    clubs = list()
    
    for td in html.select('td'):
        clubs.append(td.text)
    
    if year != '2017':
        for i in clubs[:31]:
            if 'San Jose' in i or 'New Jersey' in i or 'Tampa Bay' in i or 'Los Angeles' in i:
                club = i[6:]
            else:
                club = i[7:]
            points = clubs[clubs.index(i)+7*(5+clubs.index(i)*2)]
            if not club in nhlpAvrg.keys():
                nhlpAvrg[club] = int(points)
            else:
                nhlpAvrg[club] += int(points)
    else:
        for i in clubs[:30]:
            if 'San Jose' in i or 'New Jersey' in i or 'Tampa Bay' in i or 'Los Angeles' in i:
                club = i[6:]
            else:
                club = i[7:]
            points = clubs[clubs.index(i)+2*(17+clubs.index(i)*7)]
            if not club in nhlpAvrg.keys():
                nhlpAvrg[club] = int(points)
            else:
                nhlpAvrg[club] += int(points)
    
    
nhlPoints("2021")
nhlPoints("2020")
nhlPoints("2019")
nhlPoints("2018")
nhlPoints("2017")

for team in nhlpAvrg.keys():
    if team == 'Vegas Golden Knights':
        nhlpAvrg[team] = round(nhlpAvrg[team]/4,2)
    else:
        nhlpAvrg[team] = round(nhlpAvrg[team]/5,2)

nhlpAvrg = pd.Series(nhlpAvrg).sort_index().to_dict()
nhlpAvrg

{'Anaheim Ducks': 79.2,
 'Arizona Coyotes': 70.8,
 'Boston Bruins': 97.4,
 'Buffalo Sabres': 64.2,
 'Calgary Flames': 83.8,
 'Carolina Hurricanes': 86.0,
 'Chicago Blackhawks': 79.2,
 'Colorado Avalanche': 81.4,
 'Columbus Blue Jackets': 86.4,
 'Dallas Stars': 81.2,
 'Detroit Red Wings': 62.6,
 'Edmonton Oilers': 83.0,
 'Florida Panthers': 84.0,
 'Los Angeles Kings': 73.6,
 'Minnesota Wild': 88.4,
 'Montreal Canadiens': 80.0,
 'Nashville Predators': 90.6,
 'New Jersey Devils': 70.4,
 'New York Islanders': 85.6,
 'New York Rangers': 79.2,
 'Ottawa Senators': 68.4,
 'Philadelphia Flyers': 83.0,
 'Pittsburgh Penguins': 94.8,
 'San Jose Sharks': 82.4,
 'St. Louis Blues': 89.8,
 'Tampa Bay Lightning': 100.4,
 'Toronto Maple Leafs': 91.6,
 'Vancouver Canucks': 70.2,
 'Vegas Golden Knights': 92.5,
 'Washington Capitals': 98.8,
 'Winnipeg Jets': 88.6}

In [922]:
#1 point for first round, 2 for second round, 3 for stanley cup semifinals, 4 for stanley cup finals, 5 for winning stanley cup
nhlPlayoffs = {'Anaheim Ducks': 7,'Arizona Coyotes': 1,'Boston Bruins': 20,'Buffalo Sabres': 0,'Calgary Flames': 3,
 'Carolina Hurricanes': 10,'Chicago Blackhawks': 2,'Colorado Avalanche': 10,'Columbus Blue Jackets': 6,'Dallas Stars': 13,
 'Detroit Red Wings': 0,'Edmonton Oilers': 4,'Florida Panthers': 1,'Los Angeles Kings': 1,'Minnesota Wild': 3,
 'Montreal Canadiens': 12,'Nashville Predators': 15,'New Jersey Devils': 1,'New York Islanders': 15,'New York Rangers': 3,
 'Ottawa Senators': 6,'Philadelphia Flyers': 4,'Pittsburgh Penguins': 20,'San Jose Sharks': 10,'St. Louis Blues': 20,
 'Tampa Bay Lightning': 37,'Toronto Maple Leafs': 4,'Vancouver Canucks': 3,'Vegas Golden Knights': 23,'Washington Capitals': 21,
 'Winnipeg Jets': 10}

In [923]:
nhl = pd.DataFrame({'Average Annual Payroll Spending Over 5 years':pd.Series(nhlPayroll),
                    'Average Points Over 5 years':pd.Series(nhlpAvrg),
                    'Playoff Points Over 5 years':pd.Series(nhlPlayoffs)})

#plots:
# nhl.plot(x='Average Annual Payroll Spending Over 5 years',y='Average Points Over 5 years',kind='scatter')
# plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\NHLPoints.png")

# nhl.plot(x='Average Annual Payroll Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter')
# plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\NHLPlayoffs.png")


# #nhl
# nhl.to_excel(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\NHL.xlsx")

nhl.corr()

,Average Annual Payroll Spending Over 5 years,Average Points Over 5 years,Playoff Points Over 5 years
Average Annual Payroll Spending Over 5 years,1.000000,0.368600,0.353173
Average Points Over 5 years,0.368600,1.000000,0.754118
Playoff Points Over 5 years,0.353173,0.754118,1.000000


Conclusions:
- NHL is somewhere between NFL and NBA. Spending does have a noticeable impact on success, but it is not great.
- Despite NHL having a hard cap, there is a noticeable discrepancy between spending levels of teams.

### Major League Baseball

In [924]:
#total payroll spending over 5 years
raw_html = get("http://www.stevetheump.com/Payrolls.htm#2020_payrolls",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
clubs = list()
for td in html.select('td'):
    clubs.append(td.text)

del clubs[:265] #2021 
del clubs[762:] #pre-2016


mlbPayroll = dict()

mlb20 = clubs[:211]
del mlb20[111:113] #average for league
for i in mlb20:
    if mlb20.index(i) % 7 == 0:
        payroll = mlb20[mlb20.index(i)+5]
        mlbPayroll[i.strip('\n')] = float(payroll[1:].replace(',',''))/1000000
    

mlb19 = clubs[217:367]
del mlb19[64]

for i in mlb19:
    if mlb19.index(i)%5==0:
        payroll1 = mlb19[mlb19.index(i)+2][1:].replace(',','')
        payroll2 = mlb19[mlb19.index(i)+3][1:].replace(',','')
        mlbPayroll[i] += (float(payroll1)+float(payroll2))/1000000
        

mlb18 = clubs[368:457]
for i in mlb18:
    if mlb18.index(i)%3==0:
        payroll = mlb18[mlb18.index(i)+1]
        mlbPayroll[i] += float(payroll[1:-1])


mlb17 = clubs[463:612]
mlb17
for i in mlb17:
    if mlb17.index(i)%5==0:
        payroll = mlb17[mlb17.index(i)+3]
        if i == 'Los Angeles Angels of Anaheim':
            i = 'Los Angeles Angels'
        mlbPayroll[i] += float(payroll[1:].replace(',',''))/1000000


mlb16 = clubs[613:]
for i in mlb16:
    if mlb16.index(i)%5==0:
        payroll = mlb16[mlb16.index(i)+1].strip()
        
        #New York Yankees vs Yankees
        for key in mlbPayroll.keys():
            if i.strip() in key:
                mlbPayroll[key] += float(payroll[2:].replace(',',''))/1000000

for key in mlbPayroll.keys():
    mlbPayroll[key] = round(mlbPayroll[key],2)
mlbPayroll

{'New York Yankees': 900.49,
 'Los Angeles Dodgers': 909.43,
 'Boston Red Sox': 905.02,
 'Houston Astros': 595.02,
 'New York Mets': 631.87,
 'Chicago Cubs': 754.03,
 'Philadelphia Phillies': 556.59,
 'San Francisco Giants': 796.8,
 'St. Louis Cardinals': 662.59,
 'San Diego Padres': 424.15,
 'Los Angeles Angels': 710.09,
 'Washington Nationals': 743.69,
 'Atlanta Braves': 500.25,
 'Arizona Diamondbacks': 468.62,
 'Texas Rangers': 633.92,
 'Colorado Rockies': 576.83,
 'Cincinnati Reds': 491.43,
 'Chicago White Sox': 409.73,
 'Toronto Blue Jays': 580.64,
 'Minnesota Twins': 510.32,
 'Seattle Mariners': 627.27,
 'Detroit Tigers': 646.95,
 'Milwaukee Brewers': 412.92,
 'Cleveland Indians': 502.18,
 'Kansas City Royals': 515.94,
 'Oakland Athletics': 357.39,
 'Tampa Bay Rays': 304.52,
 'Miami Marlins': 394.96,
 'Pittsburgh Pirates': 373.92,
 'Baltimore Orioles': 518.96}

In [925]:
#average WPCT over 5 years
mlbpAvrg = dict()
def mlbWPCT(year):
    raw_html = get("https://www.espn.com/mlb/standings/_/season/"+year+"/group/overall")
    html = BeautifulSoup(raw_html.content, 'html.parser')
    clubs = list()
    
    for td in html.select('td'):
        clubs.append(td.text)

    for i in clubs[:30]:
        if '--' in i:
            if 'San Francisco' in i or 'Kansas City' in i or 'Tampa Bay' in i or 'San Diego' in i:
                club = i[6:]
            else:
                club = i[7:]
        else:
            if 'San Francisco' in i or 'Kansas City' in i or 'Tampa Bay' in i or 'San Diego' in i:
                club = i[2:]
            else:
                club = i[3:]
        
        points = clubs[clubs.index(i)+2*(5*clubs.index(i)+16)]
        if not club in mlbpAvrg.keys():
            mlbpAvrg[club] = float(points)
        else:
            mlbpAvrg[club] += float(points)

mlbWPCT("2020")
mlbWPCT("2019")
mlbWPCT("2018")
mlbWPCT("2017")
mlbWPCT("2016")

for team in mlbpAvrg.keys():
    mlbpAvrg[team] = round(mlbpAvrg[team]/5,3)
mlbpAvrg

{'Los Angeles Dodgers': 0.628,
 'Tampa Bay Rays': 0.546,
 'San Diego Padres': 0.463,
 'Minnesota Twins': 0.519,
 'Oakland Athletics': 0.537,
 'Atlanta Braves': 0.521,
 'Chicago White Sox': 0.462,
 'Cleveland Indians': 0.587,
 'Chicago Cubs': 0.575,
 'New York Yankees': 0.577,
 'Toronto Blue Jays': 0.483,
 'St. Louis Cardinals': 0.533,
 'Cincinnati Reds': 0.447,
 'Miami Marlins': 0.445,
 'Milwaukee Brewers': 0.521,
 'Houston Astros': 0.584,
 'San Francisco Giants': 0.468,
 'Philadelphia Phillies': 0.461,
 'Seattle Mariners': 0.486,
 'Kansas City Royals': 0.43,
 'Los Angeles Angels': 0.464,
 'Colorado Rockies': 0.486,
 'New York Mets': 0.482,
 'Washington Nationals': 0.54,
 'Baltimore Orioles': 0.41,
 'Arizona Diamondbacks': 0.49,
 'Boston Red Sox': 0.547,
 'Detroit Tigers': 0.403,
 'Texas Rangers': 0.466,
 'Pittsburgh Pirates': 0.44}

In [926]:
#1 point for reaching Wildcard Series, 2 points for division series, 3 points for league championship series, 4 points for world series, 5 for winning world series
#Teams playing in division series without playing wildcard get +3 points, not +2
mlbPlayoffs = {'Arizona Diamondbacks': 3,'Atlanta Braves': 12,'Baltimore Orioles': 1,'Boston Red Sox': 21,
'Chicago Cubs': 23,'Chicago White Sox': 1,'Cincinnati Reds': 1,'Cleveland Indians': 17,'Colorado Rockies': 4,
'Detroit Tigers': 0,'Houston Astros': 37,'Kansas City Royals': 0,'Los Angeles Angels': 0,'Los Angeles Dodgers': 44,
'Miami Marlins': 3,'Milwaukee Brewers': 8,'Minnesota Twins': 5,'New York Mets': 1,'New York Yankees': 18,
'Oakland Athletics': 5,'Philadelphia Phillies': 0,'Pittsburgh Pirates': 0,'San Diego Padres': 3,'San Francisco Giants': 3,
'Seattle Mariners': 0,'St. Louis Cardinals': 7,'Tampa Bay Rays': 13,'Texas Rangers': 3,'Toronto Blue Jays': 7,'Washington Nationals': 21}

In [927]:
mlb = pd.DataFrame({'Total Payroll Spending Over 5 years':pd.Series(mlbPayroll),
                    'Average WPCT Over 5 years':pd.Series(mlbpAvrg),
                    'Playoff Points Over 5 years':pd.Series(mlbPlayoffs)})

#mlb
#mlb.to_excel(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\MLB.xlsx")
mlb.corr()


#plots:
#mlb.plot(x='Total Payroll Spending Over 5 years',y='Average WPCT Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\MLBWPCT.png")
#mlb.plot(x='Total Payroll Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\MLBPlayoffs.png")

,Total Payroll Spending Over 5 years,Average WPCT Over 5 years,Playoff Points Over 5 years
Total Payroll Spending Over 5 years,1.000000,0.375362,0.486113
Average WPCT Over 5 years,0.375362,1.000000,0.857976
Playoff Points Over 5 years,0.486113,0.857976,1.000000


### Serie A

In [928]:
#transfer spending
raw_html = get("https://www.transfermarkt.us/serie-a/fuenfjahresvergleich/wettbewerb/IT1",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')

clubs = list()
for td in html.select('td'):
    clubs.append(td.text)
del clubs[:19]
del clubs[159:] #useless fluff

serieAtr = dict() #total spending for every club over the past 5 years, in millions USD
for e in clubs:
    if clubs.index(e) % 8 == 0:
        spending = clubs[clubs.index(e)+6]
        if '-' in spending:
            serieAtr[e] = float(spending[2:len(spending)-1]) #turn '$-50m' into 50
        else:
            serieAtr[e] = -float(spending[1:len(spending)-1])

#transfermarkt has data for the clubs that are new to serie a for 2021-22 season, but omits clubs that left the league in the most recent season
#as a result, I manually removed new arrivals and added the old teams (using https://www.transfermarkt.co.in/vergleich-fc-empoli-inter-mailand-inter/vereinsvergleich/statistik/749_46)  

serieAtr['Benevento'] = 40.25
serieAtr['Crotone'] = 3
serieAtr['Parma'] = 158

del serieAtr['FC Empoli']
del serieAtr['US Salernitana 1919']
del serieAtr['Venezia FC']
            
serieAtr = pd.Series(serieAtr).sort_index().to_dict()
serieAtr

{'AC Milan': 382.94,
 'ACF Fiorentina': 45.95,
 'AS Roma': -7.98,
 'Atalanta BC': -56.24,
 'Benevento': 40.25,
 'Bologna FC 1909': 66.42,
 'Cagliari Calcio': 54.34,
 'Crotone': 3.0,
 'Genoa CFC': -60.41,
 'Hellas Verona': 3.48,
 'Inter Milan': 152.22,
 'Juventus FC': 267.72,
 'Parma': 158.0,
 'SS Lazio': -1.39,
 'SSC Napoli': 155.05,
 'Spezia Calcio': -2.26,
 'Torino FC': 3.85,
 'UC Sampdoria': -50.35,
 'US Sassuolo': 9.68,
 'Udinese Calcio': -59.04}

In [929]:
#payroll spending
raw_html = get("https://www.rossoneriblog.com/2020/10/10/milan-2020-2021-players-salary-chart/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')

ser21 = list()
for td in html.select('td'):
    ser21.append(td.text)
del ser21[:162]
serSal21 = dict() #2020-21
for i in ser21:
    if ser21.index(i) % 6 == 0:
        payroll = ser21[ser21.index(i)+1]
        serSal21[i] = round(float(payroll.replace(',',''))/1000000*1.19,2) #*1.19 to get from euro to usd, rate accurate as of 7/3/21


raw_html = get("https://www.rossoneriblog.com/2019/09/10/milan-2019-2020-players-salary-chart/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
ser20 = list()
for td in html.select('td'):
    ser20.append(td.text)
del ser20[:150]
serSal20 = dict() #2019-20
for i in ser20:
    if ser20.index(i) % 6 == 0:
        payroll = ser20[ser20.index(i)+1]
        serSal20[i] = round(float(payroll.replace(',',''))/1000000*1.19,2)


raw_html = get("https://www.rossoneriblog.com/2018/09/04/milan-2018-2019-players-salary-chart/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
ser19 = list()
for td in html.select('td'):
    ser19.append(td.text)
del ser19[:174]
serSal19 = dict() #2018-19
for i in ser19:
    if ser19.index(i) % 6 == 0:
        payroll = ser19[ser19.index(i)+1]
        serSal19[i] = round(float(payroll.replace(',',''))/1000000*1.19,2)


raw_html = get("https://www.rossoneriblog.com/2017/09/07/milan-20172018-players-salary-chart/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
ser18 = list()
for td in html.select('td'):
    ser18.append(td.text)
del ser18[:168]
serSal18 = dict() #2017-18
for i in ser18:
    if ser18.index(i) % 6 == 0:
        payroll = ser18[ser18.index(i)+1]
        serSal18[i] = round(float(payroll.replace(',',''))/1000000*1.19,2)


raw_html = get("https://www.rossoneriblog.com/2016/09/07/milan-20162017-players-salary-chart/",headers = {"User-Agent":"Mozilla/5.0"})
html = BeautifulSoup(raw_html.content, 'html.parser')
ser17 = list()
for td in html.select('td'):
    ser17.append(td.text)
del ser17[120:]
serSal17 = dict() #2016-17
for i in ser17:
    if ser17.index(i) % 6 == 0:
        payroll = ser17[ser17.index(i)+1]
        serSal17[i] = round(float(payroll.replace(',',''))/1000000*1.19,2)

#combining into dataframe to calculate averages
df = pd.DataFrame({'2016-17':pd.Series(serSal17),'2017-18':pd.Series(serSal18),'2018-19':pd.Series(serSal19),'2019-20':pd.Series(serSal20),'2020-21':pd.Series(serSal21)})
df['Average'] = df.mean(axis=1)
sPayrollAvrg = dict()
for ind,row in df.iterrows():
    if not pd.isnull(row['2020-21']): #if team is currently in EPL
        name = ind
        if name == 'Milan':
            name = 'AC Milan'
        elif name == 'Roma':
            name = 'AS Roma'
        elif name == 'Fiorentina':
            name = 'ACF Fiorentina'
        elif name == 'Samdproia':
            name = 'UC Sampdoria'
        elif name == 'Sassuolo':
            name = 'US Sassuolo'
        elif name == 'Lazio':
            name = 'SS Lazio'
        elif name == 'Napoli':
            name = 'SSC Napoli'
        sPayrollAvrg[name]=round(row['Average'],2)
sPayrollAvrg = pd.Series(sPayrollAvrg).sort_index().to_dict()
sPayrollAvrg

{'AC Milan': 129.0,
 'ACF Fiorentina': 52.6,
 'AS Roma': 123.76,
 'Atalanta': 37.27,
 'Benevento': 27.96,
 'Bologna': 43.08,
 'Cagliari': 38.32,
 'Crotone': 19.44,
 'Genoa': 39.51,
 'Hellas Verona': 28.16,
 'Inter': 144.23,
 'Juventus': 251.8,
 'Parma': 34.91,
 'SS Lazio': 80.44,
 'SSC Napoli': 109.0,
 'Spezia': 26.18,
 'Torino': 53.55,
 'UC Sampdoria': 40.46,
 'US Sassuolo': 37.37,
 'Udinese': 32.37}

In [930]:
#average points over last 5 years
serieApoints = dict()
yearsinSer = dict() #to keep track of relegations and promotions
def serPoints(year):
    raw_html = get("https://www.skysports.com/serie-a-table/"+year)
    html = BeautifulSoup(raw_html.content, 'html.parser')
    clubs = list()
    
    for td in html.select('td'):
        clubs.append(td.text)

    for e in clubs:
        if clubs.index(e) % 11 == 1:
            points = clubs[clubs.index(e)+8]
            name = e.strip('\n')
            if name == 'Roma':
                name = 'AS Roma'
            elif name == 'Fiorentina':
                name = 'ACF Fiorentina'
            elif name == 'Samdproia':
                name = 'UC Sampdoria'
            elif name == 'Sassuolo':
                name = 'US Sassuolo'
            elif name == 'Lazio':
                name = 'SS Lazio'
            elif name == 'Napoli':
                name = 'SSC Napoli'
            elif name == 'Verona':
                name = 'Hellas Verona'
            if year=='2020':
                serieApoints[name] = int(points)
                yearsinSer[name] = 1
            else:
                try:
                    yearsinSer[name] += 1
                    serieApoints[name] += int(points)
                except KeyError:
                    pass #team wasn't in serie a in 2020-21, not interested in it
        
    
serPoints('2020')
serPoints('2019')
serPoints('2018')
serPoints('2017')
serPoints('2016')

for team in serieApoints.keys():
    serieApoints[team] = round(serieApoints[team]/yearsinSer[team],2)
serieApoints = pd.Series(serieApoints).sort_index().to_dict()
serieApoints

{'AC Milan': 68.0,
 'ACF Fiorentina': 49.4,
 'AS Roma': 72.4,
 'Atalanta': 71.4,
 'Benevento': 27.0,
 'Bologna': 42.4,
 'Cagliari': 41.8,
 'Crotone': 30.67,
 'Genoa': 39.2,
 'Hellas Verona': 39.67,
 'Inter Milan': 75.2,
 'Juventus': 87.4,
 'Parma': 36.67,
 'SS Lazio': 69.4,
 'SSC Napoli': 79.0,
 'Sampdoria': 49.8,
 'Spezia': 39.0,
 'Torino': 49.4,
 'US Sassuolo': 49.0,
 'Udinese': 42.6}

In [931]:
#Champions League: 1 for reaching round of 16, 2 for reaching quarterfinals, 3 for reaching semifinals, 4 for reaching finals, 5 for winning
#Coppa Italia: 1 for reaching round of 16, 2 for reaching quarterfinals, 3 for reaching semifinals, 4 for reaching finals, 5 for winning
#Europa League: 1 for reaching quarterfinals, 2 for reaching semifinals, 3 for reaching finals, 4 for winning

serPlayoffs = {'AC Milan': 28,'ACF Fiorentina': 16,'AS Roma': 24,'Atalanta BC': 32,'Benevento': 1,'Bologna FC 1909': 2,
'Cagliari Calcio': 3,'Crotone': 0,'Genoa CFC': 4,'Hellas Verona': 1,'Inter Milan': 27,'Juventus FC': 76,'Parma': 2,
'SS Lazio': 39,'SSC Napoli': 36,'Spezia Calcio': 4,'Torino FC': 9,'UC Sampdoria': 3,'US Sassuolo': 4,'Udinese Calcio': 2}

In [934]:
#putting it all together
#all dictionaries are sorted, but names of clubs are slightly different (e.g. Arsenal vs Arsenal FC). Therefore, index reset is required
serA = pd.DataFrame({'Total Transfer Spending Over 5 years':list(serieAtr.values()),
                    'Average Annual Payroll Spending Over 5 years':list(sPayrollAvrg.values()),
                    'Average Season Points Over 5 years':list(serieApoints.values()),
                    'Playoff Points Over 5 years':list(serPlayoffs.values())}, index = list(serPlayoffs.keys()))

#plots:
#serA.plot(x='Total Transfer Spending Over 5 years',y='Average Season Points Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\SERTransferPoints.png")

#serA.plot(x='Total Transfer Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter') #exponential correlation?
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\SERTransferPlayoffs.png")

#serA.plot(x='Total Transfer Spending Over 5 years',y='Average Annual Payroll Spending Over 5 years',kind='scatter')

#serA.plot(x='Average Annual Payroll Spending Over 5 years',y='Playoff Points Over 5 years',kind='scatter')
#plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\SERPayrollPlayoffs.png")

# x = np.array(serA['Average Annual Payroll Spending Over 5 years'])
# y = np.array(serA['Average Season Points Over 5 years'])
# coeffs = np.polyfit(x, y, 2)
# poly = np.poly1d(coeffs)
# new_x = np.linspace(min(x)-20,max(x))
# new_y = poly(new_x)
# serA.plot(x='Average Annual Payroll Spending Over 5 years',y='Average Season Points Over 5 years',kind='scatter')
# plt.plot(new_x,new_y,color='orange')
# plt.savefig(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\SERPayrollPoints.png")

serA.corr()

#serA
#serA.to_excel(r"C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Python\Sports spending\Project Files\SER.xlsx")

,Total Transfer Spending Over 5 years,Average Annual Payroll Spending Over 5 years,Average Season Points Over 5 years,Playoff Points Over 5 years
Total Transfer Spending Over 5 years,1.000000,0.680588,0.450261,0.522510
Average Annual Payroll Spending Over 5 years,0.680588,1.000000,0.817478,0.888259
Average Season Points Over 5 years,0.450261,0.817478,1.000000,0.890717
Playoff Points Over 5 years,0.522510,0.888259,0.890717,1.000000


Conclusions:
- Payroll has roughly the same degree of correlation with success as in EPL. However, transfer spending is less important, likely because Serie A has more teams exporting players. In EPL, for instance, every single club had a negative tranfer balance (i.e. teams spend more than they earn), whereas in Serie A plenty of teams had the opposite.
- This confirms that EPL is not just an exception - many major soccer leagues around the world have a pay-to-win system.
- Again, relationship between transfer spending and trophies.
- Payroll spending and average points have a somewhat logarithmic relationship, suggesting that throwing money on salaries helps only to a certain point.